In [91]:
import random
import numpy as np

In [83]:
# GLOBALS
ALLE_DIEREN = ['KE', 'FA', 'MU', 'NO', 'SW', 'SG', 'SA', 'HA', 'BI', 'KA', 'TU', 'AS', 'TA']
DAGDELEN = ['OCH1','OCH2','MID1','MID2']
RANDOM_SEED = 32

# 3 observaties per dagdeel mogelijk, elk dier wordt ong ~3x elk dagdeel geobserveerd
aantal_observaties = 3*len(DAGDELEN)*3*len(ALLE_DIEREN)
aantal_observaties

468

In [84]:
def generate_combinations(animal_names, timeslots):    
    
    combs = []
    for name in animal_names:
        combs += list(zip([name]*len(timeslots*3),timeslots*3))
    combs *= 3

    return combs

def generate_random_seed():
    return 

In [97]:
# GENERATE RANDOM SEQUENCE OF TIMESLOT-ANIMAL COMBINATIONS
combs = generate_combinations(ALLE_DIEREN,DAGDELEN)
random.seed(RANDOM_SEED)
random.shuffle(combs)
np.savetxt('random_schedule_list.csv',combs,delimiter=',',fmt='% s')